## Milestone 2

In [15]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')

from scipy.sparse import csr_matrix


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Gabe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Gabe\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [16]:
# Importing Reviews dataframe
reviews_df = pd.read_csv("bgg-19m-reviews.csv")
reviews_df.sample(10)


,Unnamed: 0,user,rating,comment,ID,name
12769989,12769989,CalgaryGuy76,7.0,NaN,271869,Sushi Roll
6931981,6931981,SkinnyGodfather,7.0,NaN,43443,Castle Panic
6144631,6144631,branvahn,7.0,NaN,164928,Orléans
8591869,8591869,yorkrambler,6.0,NaN,126042,Nations
11250447,11250447,dakkadakka1,9.0,NaN,197070,Massive Darkness
3379210,3379210,Snapcase,6.0,Nice components and a ton of factions mixed wi...,120677,Terra Mystica
16122289,16122289,Coffeedemon,8.0,NaN,154875,Silent Victory: U.S. Submarines in the Pacific...
1414665,1414665,lordvheod,7.0,NaN,157969,Sheriff of Nottingham
18271425,18271425,helt570,6.0,NaN,27291,AtmosFear: Khufu – The Mummy
2276803,2276803,CodGod,7.0,NaN,41114,The Resistance


In [17]:
# dropping extra index column and rows without comments
reviews_df = reviews_df.dropna(subset=['comment'])
reviews_df = reviews_df.drop(axis=1, columns="Unnamed: 0")
# creating sentiment column based on mean scores found in EDA
reviews_df['sentiment'] = reviews_df.apply(lambda row: row.rating >= 6.4, axis = 1)
reviews_df.sample(10)

,user,rating,comment,ID,name,sentiment
13214171,_Marty_,8.50,"This is a great, yet complex, CCG. With innov...",1270,Star Wars Customizable Card Game,True
12288100,Ehop16,6.00,"Quick length, two player, decent filler game (...",247367,"Air, Land & Sea",False
12908801,heffernan,10.00,Yabba dabba doooo!!!!!,230,Merchant of Venus,True
6345628,EternalReaper,10.00,Great drafting element + components + art. Won...,209010,Mechs vs. Minions,True
1332501,Casual_Offender,8.00,"Pretty light, quick, and fun. The kind of game...",50,Lost Cities,True
16441224,NKOUK7,7.00,Too much luck driven I think. If you don't pic...,11081,Familienbande,True
2739306,GlennG,5.50,"Yeah, overrated. Does have a nice Magic Realm...",96848,Mage Knight Board Game,False
6963513,emspace,7.77,Every bit as card-draw-timing dependant as I h...,286096,Tapestry,True
3848599,skorasaurus,6.00,somewhat easy to learn; but just found myself ...,2651,Power Grid,False
18197399,Martin Ralya,5.00,"Like similar games I've played, this isn't so ...",9091,The Newlywed Game,False


In [18]:
# filtering out comment elements and prepping for vectorization
cv_matrix = reviews_df.comment.str.lower()
cv_matrix = cv_matrix.apply(lambda x: re.sub('[^A-Za-z0-9]', ' ', x))
# removing stopwords and joining to apply Stemming
stop_words = stopwords.words('english')
cv_matrix = cv_matrix.apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

# stemming words and joining again, last step before vectorizer
porter = PorterStemmer()
cv_matrix = cv_matrix.apply(word_tokenize)
cv_matrix = cv_matrix.apply(lambda x: [porter.stem(word) for word in x])
cv_matrix = cv_matrix.apply(lambda x: ' '.join([word for word in x]))

cv_matrix.head(10)

1     hand favorit new game bgg con 2007 play 5 time...
2     tend either love easili tire co op game pandem...
4     amaz co op game play mostli wife game realli i...
5     hey final rate game playtest coupl year realli...
8            love great fun son 2 play far look forward
9     fun fun game strategi requir definit access no...
11    know balanc thing absolut amaz ball find new w...
12               best collabor multi player game period
13    25 play play famili game 2 5 player final coop...
16    themat brisk game well balanc creat palpabl te...
Name: comment, dtype: object

In [19]:
# Bag of words output is too large, limmited max features to 100000 and used int8
# will probably have to adjust this before implimenting the model.
vectorizer = CountVectorizer(max_features=10000)
reviews_vectored = vectorizer.fit_transform(cv_matrix).astype(np.uint8)
print(reviews_vectored.toarray().shape)

(3368619, 10000)


In [20]:
board_games_df = pd.read_csv("games_detailed_info.csv")
board_games_df.sample(10)

C:\Users\Gabe\AppData\Local\Temp\ipykernel_16976\2692140945.py:1: DtypeWarning: Columns (29,48) have mixed types. Specify dtype option on import or set low_memory=False.
  board_games_df = pd.read_csv("games_detailed_info.csv")


,Unnamed: 0,type,id,thumbnail,image,primary,alternate,description,yearpublished,minplayers,...,War Game Rank,Customizable Rank,Children's Game Rank,RPG Item Rank,Accessory Rank,Video Game Rank,Amiga Rank,Commodore 64 Rank,Arcade Rank,Atari ST Rank
16906,16906,boardgame,9153,https://cf.geekdo-images.com/F4mozadd9ZmauNck0...,https://cf.geekdo-images.com/F4mozadd9ZmauNck0...,Monopoly: NASCAR,NaN,"The classic Property Trading Game, themed to N...",1997,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5739,5739,boardgame,233015,https://cf.geekdo-images.com/HUTKETQwa4XQH4keM...,https://cf.geekdo-images.com/HUTKETQwa4XQH4keM...,Imperius,NaN,The ruling house of the empire is in decline &...,2018,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,11,boardgame,148228,https://cf.geekdo-images.com/rwOMxx4q5yuElIvo-...,https://cf.geekdo-images.com/rwOMxx4q5yuElIvo-...,Splendor,"['Goharneshan (گوهرنشان)', 'Розкіш', 'Роскошь'...",Splendor is a game of chip-collecting and card...,2014,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20888,20888,boardgame,180916,https://cf.geekdo-images.com/5uTGw3JPMoRWI-ezg...,https://cf.geekdo-images.com/5uTGw3JPMoRWI-ezg...,Saving Time,NaN,Saving Time is a cooperative time travel game ...,2019,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11158,11158,boardgame,209926,https://cf.geekdo-images.com/-cl6uHD9mzsx6n2md...,https://cf.geekdo-images.com/-cl6uHD9mzsx6n2md...,Movable Type,NaN,Description from the publisher:&#10;&#10;Movab...,2016,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5480,5480,boardgame,27739,https://cf.geekdo-images.com/MZ_yCBTfXnbv1vRH3...,https://cf.geekdo-images.com/MZ_yCBTfXnbv1vRH3...,Hearts and Minds: Vietnam 1965-1975,NaN,Hearts and Minds: Vietnam 1965-1975 is a card-...,2010,2,...,251.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9645,9645,boardgame,308493,https://cf.geekdo-images.com/zE-9tRCxmYUMUxjhe...,https://cf.geekdo-images.com/zE-9tRCxmYUMUxjhe...,Relics of Rajavihara,NaN,You&rsquo;ve discovered an ancient palace fill...,2021,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
52,52,boardgame,37111,https://cf.geekdo-images.com/5Q2w2rFJiFI_uV89K...,https://cf.geekdo-images.com/5Q2w2rFJiFI_uV89K...,Battlestar Galactica: The Board Game,"['Battlestar Galactica: Das Brettspiel', 'Batt...",Battlestar Galactica: The Board Game is an exc...,2008,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16284,16284,boardgame,5279,https://cf.geekdo-images.com/Q5IP3AWsMcMwkPLCE...,https://cf.geekdo-images.com/Q5IP3AWsMcMwkPLCE...,Sagunto: The Battle for Valencia,NaN,The Battle of Sagunto is a game of tactical si...,1993,2,...,1580.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14199,14199,boardgame,274557,https://cf.geekdo-images.com/OeMXfFtJmZ5Qes8yE...,https://cf.geekdo-images.com/OeMXfFtJmZ5Qes8yE...,Match Up! Travel,"['Kapcsolj!: A Föld körül', 'Links Voyage', 'L...",Match Up! Travel &mdash; first released as Lin...,2018,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
# dropping extra index column and comlumns that are outside of the scope of the model, such as summary statistics and alternate rankings.
board_games_df = board_games_df.drop(axis=1, columns="Unnamed: 0")
board_games_df = board_games_df.drop(columns=[ 'thumbnail', 'boardgameexpansion', 'boardgameimplementation', 'bayesaverage','Strategy Game Rank', 'Family Game Rank', 'stddev', 'median', 'numweights',
       'averageweight', 'boardgameintegration', 'boardgamecompilation',
       'Party Game Rank', 'Abstract Game Rank', 'Thematic Rank',
       'War Game Rank', 'Customizable Rank', "Children's Game Rank",
       'RPG Item Rank', 'Accessory Rank', 'Video Game Rank', 'Amiga Rank',
       'Commodore 64 Rank', 'Arcade Rank', 'Atari ST Rank', 'suggested_num_players', 'image'])
board_games_df.columns

Index(['type', 'id', 'primary', 'alternate', 'description', 'yearpublished',
       'minplayers', 'maxplayers', 'suggested_playerage',
       'suggested_language_dependence', 'playingtime', 'minplaytime',
       'maxplaytime', 'minage', 'boardgamecategory', 'boardgamemechanic',
       'boardgamefamily', 'boardgamedesigner', 'boardgameartist',
       'boardgamepublisher', 'usersrated', 'average', 'Board Game Rank',
       'owned', 'trading', 'wanting', 'wishing', 'numcomments'],
      dtype='object')

In [22]:
stop_words = stopwords.words('english')
porter = PorterStemmer()

In [23]:
# same process to vectorize descriptions as with comments
cv_matrix_desc = board_games_df.description.str.lower()
cv_matrix_desc = cv_matrix_desc.apply(lambda x: re.sub('[^A-Za-z0-9]', ' ', str(x)))


cv_matrix_desc = cv_matrix_desc.apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

cv_matrix_desc = cv_matrix_desc.apply(word_tokenize)
cv_matrix_desc = cv_matrix_desc.apply(lambda x: [porter.stem(word) for word in x])
cv_matrix_desc = cv_matrix_desc.apply(lambda x: ' '.join([word for word in x]))

cv_matrix_desc.head(10)

0    pandem sever virul diseas broken simultan worl...
1    carcassonn tile placement game player draw pla...
2    catan formerli settler catan player tri domin ...
3    leader one 7 great citi ancient world gather r...
4    quot monarch like parent ruler small pleasant ...
5    elegantli simpl gameplay ticket ride learn 15 ...
6    codenam easi parti game solv puzzl 10 game div...
7    2400 mankind begin terraform planet mar giant ...
8    mani way 7 wonder duel resembl parent game 7 w...
9    descript boardgamenew 10 10 agricola farmer wo...
Name: description, dtype: object

In [24]:
vectorizer = CountVectorizer()
descriptions_vectored = vectorizer.fit_transform(cv_matrix_desc)
print(descriptions_vectored.toarray().shape)

(21631, 51250)


In [25]:
board_games_df

,type,id,primary,alternate,description,yearpublished,minplayers,maxplayers,suggested_playerage,suggested_language_dependence,...,boardgameartist,boardgamepublisher,usersrated,average,Board Game Rank,owned,trading,wanting,wishing,numcomments
0,boardgame,30549,Pandemic,"['EPIZOotic', 'Pandemia', 'Pandemia 10 Anivers...","In Pandemic, several virulent diseases have br...",2008,2,4,"[OrderedDict([('@value', '2'), ('@numvotes', '...","[OrderedDict([('@level', '6'), ('@value', 'No ...",...,"['Josh Cappel', 'Christian Hanisch', 'Régis Mo...","['Z-Man Games', 'Albi', 'Asmodee', 'Asmodee It...",109006,7.58896,106,168364,2508,625,9344,17305
1,boardgame,822,Carcassonne,"['Carcassonne Jubilee Edition', 'Carcassonne: ...",Carcassonne is a tile-placement game in which ...,2000,2,5,"[OrderedDict([('@value', '2'), ('@numvotes', '...","[OrderedDict([('@level', '81'), ('@value', 'No...",...,"['Doris Matthäus', 'Anne Pätzke', 'Chris Quill...","['Hans im Glück', '999 Games', 'Albi', 'Bard C...",108776,7.41837,191,161299,1716,582,7383,19263
2,boardgame,13,Catan,"['CATAN', 'Catan (Колонизаторы)', 'Catan telep...","In CATAN (formerly The Settlers of Catan), pla...",1995,3,4,"[OrderedDict([('@value', '2'), ('@numvotes', '...","[OrderedDict([('@level', '86'), ('@value', 'No...",...,"['Volkan Baga', 'Tanja Donner', 'Pete Fenlon',...","['KOSMOS', '999 Games', 'Albi', 'Asmodee', 'As...",108064,7.13598,429,167733,2018,485,5890,19402
3,boardgame,68448,7 Wonders,"['7 csoda', '7 Cudów Świata', '7 divů světa', ...",You are the leader of one of the 7 great citie...,2010,2,7,"[OrderedDict([('@value', '2'), ('@numvotes', '...","[OrderedDict([('@level', '6'), ('@value', 'No ...",...,"['Dimitri Chappuis', 'Miguel Coimbra', 'Etienn...","['Repos Production', 'ADC Blackfire Entertainm...",90021,7.73515,73,120466,1567,1010,12105,14553
4,boardgame,36218,Dominion,"['Dominion: Basisspiel', 'Dominion: In naam va...","&quot;You are a monarch, like your parents bef...",2008,2,4,"[OrderedDict([('@value', '2'), ('@numvotes', '...","[OrderedDict([('@level', '161'), ('@value', 'N...",...,"['Matthias Catrein', 'Julien Delval', 'Tomasz ...","['Rio Grande Games', '999 Games', 'Albi', 'Bar...",81582,7.61000,104,106956,2009,655,8621,13787
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21626,boardgame,296892,Sacred Rites,NaN,It has come time to honor the traditions of th...,2020,3,9,"[OrderedDict([('@value', '2'), ('@numvotes', '...","[OrderedDict([('@level', '6'), ('@value', 'No ...",...,['Kristena Derrick'],['Story Machine Games'],30,7.36667,12750,112,8,4,32,14
21627,boardgame,217378,BABEL,NaN,BABEL is a balancing game in which players mak...,2016,1,5,"[OrderedDict([('@value', '2'), ('@numvotes', '...","[OrderedDict([('@level', '106'), ('@value', 'N...",...,['Masakazu Takizawa (たきざわ まさかず)'],"['こぐま工房 (Koguma Koubou)', 'Arclight']",30,6.61333,15017,64,1,6,21,10
21628,boardgame,18063,Goldrush,"['Arizona', 'Guldfeber', 'Kultakuume']",From the rules:&#10;&#10;Players are prospecto...,1970,2,4,"[OrderedDict([('@value', '2'), ('@numvotes', '...","[OrderedDict([('@level', '101'), ('@value', 'N...",...,NaN,"['Clipper', 'Condor', 'Joker', 'Pelikan']",30,5.00000,19461,98,7,4,5,13
21629,boardgame,10052,Bongo Kongo,NaN,Each player takes a position adjacent to one o...,1989,2,4,"[OrderedDict([('@value', '2'), ('@numvotes', '...","[OrderedDict([('@level', '56'), ('@value', 'No...",...,NaN,"['Ideal', 'Interplay (Board Game Related)', 'M...",30,5.80000,17262,52,4,8,13,15


In [26]:
# creating dummy variables for board game categories
genre_df = board_games_df.filter(["id","primary", "boardgamecategory"], axis=1)
genre_df.boardgamecategory = genre_df.boardgamecategory.str.strip('[]').str.split('\s*,\s*')
genre_df_vectors = genre_df.boardgamecategory.explode().str.get_dummies().sum(level=0).add_prefix('category_')
genre_df_vectors

C:\Users\Gabe\AppData\Local\Temp\ipykernel_16976\398564488.py:4: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  genre_df_vectors = genre_df.boardgamecategory.explode().str.get_dummies().sum(level=0).add_prefix('category_')


,"category_""Children's Game""",category_'Abstract Strategy',category_'Action / Dexterity',category_'Adventure',category_'Age of Reason',category_'American Civil War',category_'American Indian Wars',category_'American Revolutionary War',category_'American West',category_'Ancient',...,category_'Transportation',category_'Travel',category_'Trivia',category_'Video Game Theme',category_'Vietnam War',category_'Wargame',category_'Word Game',category_'World War I',category_'World War II',category_'Zombies'
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21626,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21627,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21628,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
21629,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
